In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [20]:
import pandas as pd

def load_housing_data(housing_path='data'):
    train_csv_path = os.path.join(housing_path, "train.csv")
    test_csv_path = os.path.join(housing_path, "test.csv")
    return pd.read_csv(train_csv_path), pd.read_csv(test_csv_path)

train, test = load_housing_data()

In [21]:
train['TotalLivingSF'] = train['BsmtFinSF1'] + train['BsmtFinSF2'] + train['1stFlrSF'] + train['2ndFlrSF']
test['TotalLivingSF'] = test['BsmtFinSF1'] + test['BsmtFinSF2'] + test['1stFlrSF'] + test['2ndFlrSF']

train['TotalLivingSF'] = np.log(train['TotalLivingSF'])
test['TotalLivingSF'] = np.log(test['TotalLivingSF'])

In [22]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
TotalLivingSF    0.698827
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr

In [23]:
num_att = list(train.select_dtypes([np.number]).columns)

# att = ['OverallQual', 'GrLivArea']

# to_remove = ['BsmtFinSF2', 'MoSold', '3SsnPorch', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'YrSold',
#             'PoolArea', 'OverallCond', 'MSSubClass', 'ScreenPorch']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold', 'YrSold',
            'GarageArea', 'TotalBsmtSF']

to_remove = ['PoolArea', 'BsmtHalfBath', 'MiscVal', 'LowQualFinSF', 'ScreenPorch', '3SsnPorch',
            'BsmtFinSF2', 'MoSold']

for t in to_remove:
    num_att.remove(t)
num_att.remove('Id')
num_att.remove('SalePrice')

# num_att = ['OverallQual', 'GrLivArea', 'GarageCars', '1stFlrSF', 'FullBath', 'YearBuilt']

print(num_att)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'YrSold', 'TotalLivingSF']


In [24]:
test['KitchenQual'] = test['KitchenQual'].fillna("TA") 

train['FireplaceQu'] = train['FireplaceQu'].fillna("None") 
test['FireplaceQu'] = test['FireplaceQu'].fillna("None") 

In [25]:
cat_att = ['BldgType', 'CentralAir', 'Foundation', 
           'PavedDrive', 'SaleCondition']
cat_att = ['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 
           'Street', 'LotShape', 'LandContour', 'LotConfig',
           'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 
           'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 'KitchenQual',
          'FireplaceQu']

# TO ADD = Exterior1st, KitchenQual, Functional, SaleType, Alley, Condition2

# cat_att = ['Neighborhood', 'BldgType']
print(cat_att)

['Neighborhood', 'BldgType', 'Heating', 'HeatingQC', 'CentralAir', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Condition1', 'HouseStyle', 'RoofStyle', 'ExterQual', 'ExterCond', 'Foundation', 'PavedDrive', 'SaleCondition', 'KitchenQual', 'FireplaceQu']


In [26]:
for a in num_att:
    median = train[a].median()
    train[a].fillna(0, inplace=True)
    
#     median = test[a].median()
    test[a].fillna(0, inplace=True)

In [27]:
import seaborn as sns
from scipy.stats import norm
from scipy import stats

train['SalePrice'] = np.log(train['SalePrice'])
train_y = train["SalePrice"].copy()

train['GrLivArea'] = np.log(train['GrLivArea'])
test['GrLivArea'] = np.log(test['GrLivArea'])

train['HasBsmt'] = pd.Series(len(train['TotalBsmtSF']), index=train.index)
train['HasBsmt'] = 0 
train.loc[train['TotalBsmtSF']>0,'HasBsmt'] = 1
train.loc[train['HasBsmt']==1,'TotalBsmtSF'] = np.log(train['TotalBsmtSF'])
test['HasBsmt'] = pd.Series(len(test['TotalBsmtSF']), index=test.index)
test['HasBsmt'] = 0 
test.loc[test['TotalBsmtSF']>0,'HasBsmt'] = 1
test.loc[test['HasBsmt']==1,'TotalBsmtSF'] = np.log(test['TotalBsmtSF'])

train['1stFlrSF'] = np.log(train['1stFlrSF'])
test['1stFlrSF'] = np.log(test['1stFlrSF'])

train['HasBsmtFin'] = pd.Series(len(train['BsmtFinSF1']), index=train.index)
train['HasBsmtFin'] = 0 
train.loc[train['BsmtFinSF1']>0,'HasBsmtFin'] = 1
train.loc[train['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(train['BsmtFinSF1'])
test['HasBsmtFin'] = pd.Series(len(test['BsmtFinSF1']), index=test.index)
test['HasBsmtFin'] = 0 
test.loc[test['BsmtFinSF1']>0,'HasBsmtFin'] = 1
test.loc[test['HasBsmtFin']==1,'BsmtFinSF1'] = np.log(test['BsmtFinSF1'])

train['HasWoodDeck'] = pd.Series(len(train['WoodDeckSF']), index=train.index)
train['HasWoodDeck'] = 0 
train.loc[train['WoodDeckSF']>0,'HasWoodDeck'] = 1
train.loc[train['HasWoodDeck']==1,'WoodDeckSF'] = np.log(train['WoodDeckSF'])
test['HasWoodDeck'] = pd.Series(len(test['WoodDeckSF']), index=test.index)
test['HasWoodDeck'] = 0 
test.loc[test['WoodDeckSF']>0,'HasWoodDeck'] = 1
test.loc[test['HasWoodDeck']==1,'WoodDeckSF'] = np.log(test['WoodDeckSF'])


/home/marek/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [28]:
corr_matrix = train.corr()
corr_matrix["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.817184
TotalLivingSF    0.741062
GrLivArea        0.730255
GarageCars       0.680625
GarageArea       0.650888
1stFlrSF         0.608947
FullBath         0.594771
YearBuilt        0.586570
YearRemodAdd     0.565608
TotRmsAbvGrd     0.534422
Fireplaces       0.489449
MasVnrArea       0.426775
TotalBsmtSF      0.373009
GarageYrBlt      0.349014
WoodDeckSF       0.343269
OpenPorchSF      0.321053
HasWoodDeck      0.320349
2ndFlrSF         0.319300
HalfBath         0.313982
LotArea          0.257320
BsmtFullBath     0.236224
BsmtUnfSF        0.221985
BedroomAbvGr     0.209044
BsmtFinSF1       0.207962
HasBsmt          0.199634
LotFrontage      0.179303
HasBsmtFin       0.162231
ScreenPorch      0.121208
PoolArea         0.069798
MoSold           0.057329
3SsnPorch        0.054900
BsmtFinSF2       0.004832
BsmtHalfBath    -0.005149
Id              -0.017942
MiscVal         -0.020021
OverallCond     -0.036868
YrSold          -0.037263
LowQualFinSF

In [29]:
train2 = train[num_att + cat_att]
train2.info()
test2 = test[num_att + cat_att]
test2.info()

merged_df = pd.concat([train2, test2])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 49 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null float64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null float64
1stFlrSF         1460 non-null float64
2ndFlrSF         1460 non-null int64
GrLivArea        1460 non-null float64
BsmtFullBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
GarageYrBlt      1460 non-null float64
GarageCars       1460 non-null int64
GarageArea       1460 non-nu

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])
train_prepared = num_pipeline.fit_transform(train[num_att])
test_prepared  = num_pipeline.transform(test[num_att])


full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_att),
        ("cat", OneHotEncoder(), cat_att),
    ])

train_test_full_prepared = full_pipeline.fit_transform(merged_df)
train_full_prepared = full_pipeline.transform(train2)
test_full_prepared = full_pipeline.transform(test2)


In [31]:
print(train_prepared.shape)
print(test_prepared.shape)
print(train_full_prepared.shape)
print(test_full_prepared.shape)

(1460, 29)
(1459, 29)
(1460, 147)
(1459, 147)


In [32]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=350, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

housing_predictions = forest_reg.predict(train_full_prepared)

forest_mse = mean_squared_error(train_y, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

0.0014825283656595318

In [33]:
print(housing_predictions[0])
print(train_y[0])

12.247694320220951
12.247694320220994


In [34]:
train_y_normal = np.e**train_y
pred_normal = np.e**housing_predictions

forest_mse = mean_squared_error(train_y_normal, pred_normal)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

207.04757989854517

In [35]:
print(train_y_normal[0])
print(pred_normal[0])

208500.0
208499.9999999911


In [36]:
# forest_reg.feature_importances_

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(forest_reg.feature_importances_, attributes), reverse=True)

[(0.1745179525716209, 'OverallQual'),
 (0.1290524265355034, 'GrLivArea'),
 (0.11575757966679612, 'TotalLivingSF'),
 (0.0647163331535607, 'YearBuilt'),
 (0.051650418777656304, 'TA'),
 (0.045935054428071505, 'GarageArea'),
 (0.04550391351848995, 'GarageCars'),
 (0.04144591775044512, 'TotalBsmtSF'),
 (0.039088760446211554, 'FullBath'),
 (0.029407473984933527, '1stFlrSF'),
 (0.023313960801624914, 'GarageYrBlt'),
 (0.016678794997365638, 'LotArea'),
 (0.01569182993286215, 'YearRemodAdd'),
 (0.0137615102206101, 'Fireplaces'),
 (0.012732328807120406, '2ndFlrSF'),
 (0.012249657705972875, 'BsmtFinSF1'),
 (0.011076581169247379, 'None'),
 (0.010139511793300894, 'TA'),
 (0.009463320236603463, 'OverallCond'),
 (0.008912077692319486, 'Y'),
 (0.007687548986621459, 'Gd'),
 (0.007586832853775165, 'BsmtUnfSF'),
 (0.0074830060144151994, 'N'),
 (0.006943638022672408, 'TotRmsAbvGrd'),
 (0.006858547563151486, 'PConc'),
 (0.005268721790053253, 'OpenPorchSF'),
 (0.005246548686358559, 'Ex'),
 (0.004934655316452

In [37]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("scores:", scores)
    print("mean:", scores.mean())
    print("std:", scores.std())

forest_reg = RandomForestRegressor(bootstrap=False, n_estimators=450, max_features=32, random_state=42)
forest_reg.fit(train_full_prepared, train_y)

forest_scores = cross_val_score(forest_reg, train_full_prepared, train_y,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

print(pd.Series(forest_rmse_scores).describe())

scores: [0.12942884 0.11943405 0.13037082 0.1715441  0.15485459 0.10810289
 0.13451708 0.11568347 0.12463915 0.13508825]
mean: 0.13236632150259914
std: 0.017774280975484856
count    10.000000
mean      0.132366
std       0.018736
min       0.108103
25%       0.120735
50%       0.129900
75%       0.134945
max       0.171544
dtype: float64


In [40]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # sprawdza 12 (3×4) kombinacji hiperparametrów
#     {'n_estimators': [100, 200, 300, 350, 400, 450], 'max_features': [16, 32, 64]},
    # następnie sprawdza 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [100, 300, 400, 500, 600, 700, 800], 
     'max_features': [16, 32, 48]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia na pięciu podzbiorach, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error',
                           return_train_score=True, verbose=3, n_jobs=-1)
grid_search.fit(train_full_prepared, train_y)

Fitting 10 folds for each of 21 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.9s
/home/marek/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:  2.1min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'bootstrap': [False], 'max_featur

In [41]:
grid_search.best_params_

{'bootstrap': False, 'max_features': 32, 'n_estimators': 700}

In [42]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
                      max_features=32, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=700,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [43]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

0.13497812120788957 {'bootstrap': False, 'max_features': 16, 'n_estimators': 100}
0.1341782915778705 {'bootstrap': False, 'max_features': 16, 'n_estimators': 300}
0.1339514139862431 {'bootstrap': False, 'max_features': 16, 'n_estimators': 400}
0.13403946373602918 {'bootstrap': False, 'max_features': 16, 'n_estimators': 500}
0.1341869304603879 {'bootstrap': False, 'max_features': 16, 'n_estimators': 600}
0.13415613686125125 {'bootstrap': False, 'max_features': 16, 'n_estimators': 700}
0.13420605009304132 {'bootstrap': False, 'max_features': 16, 'n_estimators': 800}
0.1347692357619099 {'bootstrap': False, 'max_features': 32, 'n_estimators': 100}
0.13401825440244428 {'bootstrap': False, 'max_features': 32, 'n_estimators': 300}
0.13365076135432288 {'bootstrap': False, 'max_features': 32, 'n_estimators': 400}
0.1335685811381989 {'bootstrap': False, 'max_features': 32, 'n_estimators': 500}
0.13356293238867004 {'bootstrap': False, 'max_features': 32, 'n_estimators': 600}
0.13341752182012542 {

In [44]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([4.40085309e-03, 5.14422317e-03, 1.69330735e-02, 1.86955861e-01,
       9.45816246e-03, 6.56527441e-02, 1.46717782e-02, 4.20596504e-03,
       1.22666933e-02, 7.60749458e-03, 4.02076941e-02, 2.83892452e-02,
       1.22144778e-02, 1.28905852e-01, 1.45642711e-03, 3.39454168e-02,
       2.82982092e-03, 3.75677788e-03, 1.68759965e-03, 6.49538538e-03,
       1.28984888e-02, 2.50032762e-02, 4.44653195e-02, 4.32613942e-02,
       3.10449855e-03, 5.25603137e-03, 1.98438822e-03, 2.28647211e-03,
       1.14107257e-01, 3.67437211e-05, 5.42455341e-06, 1.17084005e-04,
       2.97176320e-04, 3.87411220e-04, 2.69880443e-04, 1.39502739e-03,
       1.32570468e-03, 1.70864870e-04, 8.77713009e-04, 3.38329270e-04,
       1.75865473e-04, 6.66339994e-04, 2.20847616e-05, 2.42837144e-04,
       2.77211101e-04, 3.16983384e-04, 1.28550407e-03, 1.53731015e-04,
       3.57900452e-04, 1.40951979e-04, 3.28656584e-04, 2.25364812e-04,
       1.30307242e-04, 4.35701220e-05, 1.16702867e-03, 1.49285380e-04,
      

In [45]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = []
for a in cat_encoder.categories_:
    for b in a:
        cat_one_hot_attribs.append(b)
# print(cat_one_hot_attribs)
attributes = num_att + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.1869558609755812, 'OverallQual'),
 (0.12890585234741944, 'GrLivArea'),
 (0.11410725686613737, 'TotalLivingSF'),
 (0.06565274410373037, 'YearBuilt'),
 (0.05154191878810931, 'TA'),
 (0.044465319542563775, 'GarageCars'),
 (0.04326139421222161, 'GarageArea'),
 (0.040207694107061584, 'TotalBsmtSF'),
 (0.03394541683779165, 'FullBath'),
 (0.028389245229517707, '1stFlrSF'),
 (0.025003276162709122, 'GarageYrBlt'),
 (0.01693307350759035, 'LotArea'),
 (0.014671778230157634, 'YearRemodAdd'),
 (0.012898488777696462, 'Fireplaces'),
 (0.012266693272612938, 'BsmtFinSF1'),
 (0.012214477764158873, '2ndFlrSF'),
 (0.011257665184399883, 'None'),
 (0.009458162455944124, 'OverallCond'),
 (0.009365191872990527, 'TA'),
 (0.009269551082500885, 'Y'),
 (0.007940393525608316, 'PConc'),
 (0.007607494583204598, 'BsmtUnfSF'),
 (0.007553384406295369, 'N'),
 (0.006826742534235945, 'Gd'),
 (0.006495385376522059, 'TotRmsAbvGrd'),
 (0.005793035116199834, 'Ex'),
 (0.005256031368475015, 'OpenPorchSF'),
 (0.0051442231654

In [46]:
# y_pred = forest_reg.predict(test_full_prepared)
y_pred = grid_search.predict(test_full_prepared)
y_pred_normal = np.e**y_pred
sub = pd.DataFrame({'Id': test['Id'], 'SalePrice': y_pred_normal})

In [47]:
sub.to_csv("data/submission_script39_FireplaceQu.csv", index=False)

In [28]:
sub.shape

(1459, 2)